In [22]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [23]:
# things we need for NLP
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

# things we need for Tensorflow
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD
import pandas as pd
import pickle
import random
import json

In [24]:
with open("/content/drive/MyDrive/Chatbot_for_mentalhealth/final-intents.json") as file:
  data = json.load(file)


words = []
classes = []
documents = []
ignore_words = ['?']
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [25]:
print(data)

{'intents': [{'tag': 'depression', 'patterns': ['I am feeling a lot depressed', 'I feel low all the time', 'I am always worried about myself', 'I always feel sad and uneasy'], 'responses': ['Attend regular counselling sessions', 'Do regular exercise'], 'context': ['']}, {'tag': 'general', 'patterns': ['I want to eat', 'I want to watch you', 'I am going to play', 'My personala info is so and so'], 'responses': ['I do not answer General Questions', 'Sorry I only answer mental health related Questions'], 'context': ['']}, {'tag': 'anxiety disorders', 'patterns': ['I am feeling anxious all the time', 'I think I am facing some anxiety disorders', 'I am feeling worried ', 'I feel tensed and restless'], 'responses': ['Get Adequate sleep', 'Avoid thinking about negative thoughts'], 'context': ['']}, {'tag': 'bipolar affective disorder', 'patterns': ['I am having extreme mood swings', 'I am facing extreme emotional highs and lows', 'I am having bipolar affective disorder', 'I think I am sufferi

In [26]:
# loop through each sentence in our intents patterns
for intent in data['intents']:
    for pattern in intent['patterns']:
        # tokenize each word in the sentence
        w = nltk.word_tokenize(pattern)
        # add to our words list
        words.extend(w)
        # add to documents in our corpus
        documents.append((w, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

# stem and lower each word and remove duplicates
words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

# sort classes
classes = sorted(list(set(classes)))

# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique stemmed words", words)

91 documents
24 classes ['Eating', 'OCD', 'Paranoia', 'affects_whom', 'anxiety disorders', 'behavioural and emotional disorders', 'bipolar affective disorder', 'causeeat', 'definition', 'depression', 'dissociation and dissociative disorders', 'eatdefine', 'find_help', 'general', 'post_traumatic_stress_disorder', 'professional_types', 'psychosis', 'recover', 'right_professional', 'schizophrenia', 'steps', 'treatement_options', 'treatment_tips', 'what_causes']
232 unique stemmed words [',', '.', 'a', 'abnorm', 'about', 'affect', 'against', 'aggress', 'al', 'alon', 'alway', 'am', 'an', 'and', 'ang', 'anxy', 'appear', 'ar', 'as', 'avail', 'avoid', 'bang', 'be', 'becom', 'begin', 'behav', 'behavio', 'behavy', 'believ', 'between', 'bipol', 'blam', 'body', 'by', 'can', 'caus', 'certain', 'chang', 'clap', 'click', 'cloudy', 'compel', 'confid', 'confus', 'contamin', 'control', 'count', 'dead', 'deal', 'delud', 'depress', 'describ', 'diff', 'disconnect', 'disord', 'disorg', 'disturb', 'do', 'doe

In [27]:
# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)

# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # stem each word - create base word, in attempt to represent related words
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)

# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])

<ipython-input-27-7ad200669451>:26: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)


In [28]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))


# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(learning_rate=0.01)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

In [29]:
# Fit the model
model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)

Epoch 1/200
19/19 [==============================] - 1s 2ms/step - loss: 3.2098 - accuracy: 0.0769
Epoch 2/200
19/19 [==============================] - 0s 2ms/step - loss: 3.1914 - accuracy: 0.0549
Epoch 3/200
19/19 [==============================] - 0s 2ms/step - loss: 3.1534 - accuracy: 0.0330
Epoch 4/200
19/19 [==============================] - 0s 2ms/step - loss: 3.2094 - accuracy: 0.0440
Epoch 5/200
19/19 [==============================] - 0s 2ms/step - loss: 3.1657 - accuracy: 0.0549
Epoch 6/200
19/19 [==============================] - 0s 2ms/step - loss: 3.1461 - accuracy: 0.0769
Epoch 7/200
19/19 [==============================] - 0s 2ms/step - loss: 3.0657 - accuracy: 0.1538
Epoch 8/200
19/19 [==============================] - 0s 2ms/step - loss: 3.0970 - accuracy: 0.1099
Epoch 9/200
19/19 [==============================] - 0s 2ms/step - loss: 3.1190 - accuracy: 0.1319
Epoch 10/200
19/19 [==============================] - 0s 2ms/step - loss: 3.0808 - accuracy: 0.1209
Epoch 11/

In [30]:
def clean_up_sentence(sentence):
    # tokenize the pattern - split words into array
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word - create short form for word
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=False):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words)
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s:
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)

    return(np.array(bag))


In [31]:
def chat():
  print("Start talking with the bot!")
  while True:
    inp = input("You: ")
    if inp.lower() == "quit":
      break
    input_data = pd.DataFrame([bow(inp, words)], dtype=float, index=['input'])
    results = model.predict([input_data])
    results_index = np.argmax(results)
    tag = classes[results_index]

    for tg in data["intents"]:
      if tg['tag'] == tag:
        responses = tg['responses']
    print(random.choice(responses))

chat()

Start talking with the bot!
You: i am depressed
1/1 [==============================] - 0s 62ms/step
Listen to music
You: i am feeling depressed
1/1 [==============================] - 0s 24ms/step
Attend regular counselling sessions
You: quit
